# Artificial Intelligence - Computer Assignment 3 - Bayesian Network

## Mehrdad Nourbakhsh(810194418)

In this project, we want to use Bayes rule for classifying news based on a short description of that news.

In [1]:
import numpy as np
import pandas as pd
import itertools
import collections
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

We have a dataset with nearly 23000 news. each news has a category. we want to create a model to classify the category of that news based on the short description of that news. we read the dataset from .csv file and load that into a panda dataframe.

In [2]:
dataFrame = pd.read_csv('Attachment/data.csv')
dataFrame.head()

,index,authors,category,date,headline,link,short_description
0,0,"Katherine LaGrave, ContributorTravel writer an...",TRAVEL,2014-05-07,"EccentriCities: Bingo Parties, Paella and Isla...",https://www.huffingtonpost.com/entry/eccentric...,Påskekrim is merely the tip of the proverbial ...
1,1,Ben Hallman,BUSINESS,2014-06-09,Lawyers Are Now The Driving Force Behind Mortg...,https://www.huffingtonpost.com/entry/mortgage-...,NaN
2,2,Jessica Misener,STYLE & BEAUTY,2012-03-12,Madonna 'Truth Or Dare' Shoe Line To Debut Thi...,https://www.huffingtonpost.com/entry/madonna-s...,"Madonna is slinking her way into footwear now,..."
3,3,"Victor and Mary, Contributor\n2Sense-LA.com",TRAVEL,2013-12-17,Sophistication and Serenity on the Las Vegas S...,https://www.huffingtonpost.com/entry/las-vegas...,But what if you're a 30-something couple that ...
4,4,"Emily Cohn, Contributor",BUSINESS,2015-03-19,It's Still Pretty Hard For Women To Get Free B...,https://www.huffingtonpost.com/entry/free-birt...,Obamacare was supposed to make birth control f...


We can see that each news has some other information too rather than the short description (e.g headline, authors). but we ignore them and use only words of the short description to train our model.

First, we should preprocess the data and use the result for the model. we want to normalize the short description.
we use nltk library for normalizing the text.normalization include these steps:
* converting uppercase to lowercase
* removing punctuation signs
* removing numbers
* converting each text into a list of words
* removing stop words
* using lemmatization to remove inflectional endings only and to return the base form of a word

If we don't convert all words to lowercase, our model might treat a word which is at the beginning of a sentence with a capital letter, different from the same word which appears later in the sentence but without any capital latter. this might influence our model accuracy. thus we convert all letters to lowercase.

In our model, we use words frequency and occurrences of them in the text. we want to find relevant results not only for the exact expression but also for the other possible forms of the words we used. for this purpose we use lemmatization. lemmatization helps us to treat all possible forms of a word as an individual word and this can improve our model with increasing word frequency for each category.

In [21]:
def cleanText(df):
    
    data = df.short_description
    data = data.fillna('')
    data = data.str.lower()
    data = data.str.replace('[^\w\s]',' ')
    data = data.str.replace('\d+', '')
    stopWords = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    wordsList = data.apply(word_tokenize)
    wordsList = wordsList.apply(lambda x: [item for item in x if item not in stopWords])
    wordsList = wordsList.apply(lambda x: [lemmatizer.lemmatize(y, pos="v") for y in x])
    df['words'] = wordsList
    return df

dataFrame = cleanText(dataFrame)
dataFrame.head()

,index,authors,category,date,headline,link,short_description,words
0,0,"Katherine LaGrave, ContributorTravel writer an...",TRAVEL,2014-05-07,"EccentriCities: Bingo Parties, Paella and Isla...",https://www.huffingtonpost.com/entry/eccentric...,Påskekrim is merely the tip of the proverbial ...,"[påskekrim, merely, tip, proverbial, iceberg, ..."
1,1,Ben Hallman,BUSINESS,2014-06-09,Lawyers Are Now The Driving Force Behind Mortg...,https://www.huffingtonpost.com/entry/mortgage-...,NaN,[]
2,2,Jessica Misener,STYLE & BEAUTY,2012-03-12,Madonna 'Truth Or Dare' Shoe Line To Debut Thi...,https://www.huffingtonpost.com/entry/madonna-s...,"Madonna is slinking her way into footwear now,...","[madonna, slink, way, footwear, truth, dare, p..."
3,3,"Victor and Mary, Contributor\n2Sense-LA.com",TRAVEL,2013-12-17,Sophistication and Serenity on the Las Vegas S...,https://www.huffingtonpost.com/entry/las-vegas...,But what if you're a 30-something couple that ...,"[something, couple, shy, away, table, dance, e..."
4,4,"Emily Cohn, Contributor",BUSINESS,2015-03-19,It's Still Pretty Hard For Women To Get Free B...,https://www.huffingtonpost.com/entry/free-birt...,Obamacare was supposed to make birth control f...,"[obamacare, suppose, make, birth, control, fre..."


We need to separate our data into two parts. the train set and evaluate set. we use 80 percent of data as train set and 20 percent as evaluate set. we want to have all sorts of news. if we choose 80 percent of real data as train set, we may have a set with only one or two categories, therefore, our model can not detect other categories as well. we have to create a train set with good diversity from all of the categories. 

for each category, we choose a random subset of that category (80 percent) and then combine those subsets to create our train set and the remaining 20 percent of each category for evaluate set.

In [4]:
travel = dataFrame[dataFrame['category'] == 'TRAVEL']
business = dataFrame[dataFrame['category'] == 'BUSINESS']
sb = dataFrame[dataFrame['category'] == 'STYLE & BEAUTY']
trainTravel = travel.sample(frac=0.8)
evaluateTravel = travel.drop(trainTravel.index)
trainBusiness = business.sample(frac=0.8)
evaluateBusiness = business.drop(trainBusiness.index)
trainSB = sb.sample(frac=0.8)
evaluateSB = sb.drop(trainSB.index)
evaluateData = pd.concat([evaluateSB,evaluateBusiness, evaluateTravel])
evaluateData = evaluateData.sample(frac=1)

After creating our train set, we should train our model with this set. 
for this purpose, we use Bayes rule.

$$ P(c|x) = \frac{P(x|c)\times P(c)}{P(x)} $$

As we can see, Bayes rule has 4 parts: Posterior, Likelihood, Prior and Evidence.in order to use this rule, we should define each part of the Bayes rule for our project.

The posterior probability is the probability of a category given the words in the news. we use Bayes rule to calculate this probability.

$$ posterior = {P(category | x_0, x_1, x_2, ...,  x_n)} $$

which is $x_n$ is the n-th word of that news.

We define the probability of each category as the prior probability which means how probable is it for a news to be for a certain category.

We define the likelihood as the probability of each word of a news given the category which means how probable it is for that category to use that word. in other words, the likelihood probability is:

$$ likelihood = {P(x_0, x_1, x_2, ...,  x_n|category)} $$

Since the probability of existing a word in a certain category is independent of the probability of existing another word in that category for each news, we can multiply these probabilities to calculate our conditional probability.

$$ likelihood = {P(x_0|category)\times P(x_1|category) \times P(x_2|category) \times ... \times P(x_n|category)} $$

The evidence is the probabiliy of all words that we have in a given news.

$$ evidence = {P(x_0, x_1, x_2, ...,  x_n)} $$

Since we are going to compare the posterior probabilities for each category and in each category the evidence probability is the same as other categories, thus we don't need to calculate the evidence probability.



##  Part $\mathrm{I}$

In this part, we want to train our model for only two categories. TRAVEL category and BUSINESS category.

In [5]:
travelWords = trainTravel.words.values
travelWords = list(itertools.chain.from_iterable(travelWords))
BusinessWords = trainBusiness.words.values
BusinessWords = list(itertools.chain.from_iterable(BusinessWords))
allTravelAndBusinessWord = list(map(''.join, set(itertools.chain(travelWords, BusinessWords))))
travelWordsCount = dict(collections.Counter(travelWords))
BusinessWordsCount = dict(collections.Counter(BusinessWords))
newDataFrame = pd.DataFrame(columns=['Word','Travel occurrences','Business occurrences'])
for word in allTravelAndBusinessWord:
    to = 0
    bo = 0
    if word in travelWordsCount:
        to = travelWordsCount[word]
    if word in BusinessWordsCount:
        bo = BusinessWordsCount[word]
    newDataFrame = newDataFrame.append({'Word' : word,'Travel occurrences' : to,'Business occurrences' : bo},ignore_index=True)


We create a new data frame. for each word, we calculate the probability of that word given the category. in fact, we calculate $ P(word | category) $ for each word in our training set (in this case out training set contain only two mentioned categories).


For calculating the conditional probability we use the Laplace Smoothing. since we multiply the conditional probabilities in order to calculate the likelihood, if we have a word that used only one time in one category, the likelihood probability for other categories will be zero even if other conditional probabilities have a high value. Laplace smoothing is used to solve the problem of zero probability.

$$ P(word|category) =  \frac{O(word,category) + \alpha}{S(words,categor) + |A|\alpha} $$

We use this formula to calculate the conditional probability. O(word, category) is the word occurrences in that category, S(words, category) is the number of all words in that category and |A| is the number of distinct words in our train set.
alpha is a constant which is used to solve zero probability problem.

In [6]:
alpha = 0.5
newDataFrame['Travel Probability'] = (newDataFrame['Travel occurrences'] + alpha) / (newDataFrame['Travel occurrences'].sum() + (len(set(travelWords + BusinessWords))*alpha))
newDataFrame['Business Probability'] = (newDataFrame['Business occurrences'] + alpha) / (newDataFrame['Business occurrences'].sum() + (len(set(travelWords + BusinessWords))*alpha))
newDataFrame = newDataFrame.set_index('Word')
newDataFrame

,Travel occurrences,Business occurrences,Travel Probability,Business Probability
Word,,,,
lighter,2,0,2.23313e-05,8.168e-06
join,25,25,0.000227779,0.000416568
família,2,0,2.23313e-05,8.168e-06
bellavista,1,0,1.33988e-05,8.168e-06
luminous,2,0,2.23313e-05,8.168e-06
...,...,...,...,...
leary,1,0,1.33988e-05,8.168e-06
fairness,1,3,1.33988e-05,5.7176e-05
whoever,3,0,3.12638e-05,8.168e-06


In [7]:
travelAndBusinessEvaluateData = pd.concat([evaluateBusiness, evaluateTravel])
travelAndBusinessEvaluateData = travelAndBusinessEvaluateData.sample(frac=1)

After calculating the conditional probability our model is ready to test with evaluate data. for each news in evaluate set, we calculate the prior probability and for all of the words in that news, we multiply the conditional probability with prior probability in order to calculate the posterior probability. After that, we can predict the category for each news. the category with higher posterior probability is our model prediction.

In [8]:
for index,row in travelAndBusinessEvaluateData.iterrows():
    travelPriorProbability = len(trainTravel)/(len(trainTravel)+len(trainBusiness))
    businessPriorProbability = len(trainBusiness)/(len(trainTravel)+len(trainBusiness))
    words = set(row['words'])
    for word in words:
        if word in allTravelAndBusinessWord:
            travelPriorProbability *= newDataFrame.at[word,'Travel Probability']
            businessPriorProbability *= newDataFrame.at[word,'Business Probability']

    travelAndBusinessEvaluateData.at[index,'Travel Probability'] = travelPriorProbability
    travelAndBusinessEvaluateData.at[index,'Business Probability'] = businessPriorProbability
    if travelPriorProbability >= businessPriorProbability:
        travelAndBusinessEvaluateData.at[index,'Prediction'] = 'TRAVEL'
    else:
        travelAndBusinessEvaluateData.at[index,'Prediction'] = 'BUSINESS'
travelAndBusinessEvaluateData.head()

,index,authors,category,date,headline,link,short_description,words,Travel Probability,Business Probability,Prediction
2452,2452,"Peter Fuda, ContributorPrincipal, The Alignmen...",BUSINESS,2014-07-07,Embed Routines and Rituals (Principle No. 5 of...,https://www.huffingtonpost.com/entry/embed-rou...,NaN,[],6.248244e-01,3.751756e-01,TRAVEL
2130,2130,"24/7 Wall St., 24/7 Wall St.",BUSINESS,2013-03-23,10 Banks Foreclosing on the Most Homes: 24/7 W...,https://www.huffingtonpost.com/entry/bank-fore...,Despite not being directly on the hook for the...,"[despite, directly, hook, majority, losses, in...",4.855343e-42,4.853664e-38,BUSINESS
12749,12749,Jamie Feldman,TRAVEL,2016-03-22,The Blooming Cherry Blossoms In Japan Look Dow...,https://www.huffingtonpost.com/entry/cherry-bl...,Time to book a flight.,"[time, book, flight]",5.783270e-09,5.695067e-11,TRAVEL
3182,3182,"Christopher Elliott, Contributor\nAuthor, How ...",TRAVEL,2013-04-25,Could Your Son be Next?,https://www.huffingtonpost.com/entry/tsa-secur...,Do we really have to trade our dignity for sec...,"[really, trade, dignity, security, tsa, trauma...",7.043141e-53,2.032097e-57,TRAVEL
17058,17058,"Shelley Miller, Contributor\nHome Exchange Expert",TRAVEL,2012-04-27,A Virtually Free Vacation Can Be Yours When Yo...,https://www.huffingtonpost.com/entry/a-virtual...,Suppose I told you that you can afford a vacat...,"[suppose, tell, afford, vacation, even, travel...",3.364800e-52,2.051141e-60,TRAVEL


In [9]:
#Travel
correctTravel = travelAndBusinessEvaluateData.loc[(travelAndBusinessEvaluateData['category'] == 'TRAVEL') & (travelAndBusinessEvaluateData['Prediction'] == 'TRAVEL')]
correctTravel = (correctTravel.all(axis='columns')).sum()
allOfTravel = (travelAndBusinessEvaluateData['category'] == 'TRAVEL').sum()
allTravelPrediction = (travelAndBusinessEvaluateData['Prediction'] == 'TRAVEL').sum()
travelRecall = correctTravel/allOfTravel
travelPrecision = correctTravel/allTravelPrediction
print(travelRecall,travelPrecision)
#Business
correctBusiness = travelAndBusinessEvaluateData.loc[(travelAndBusinessEvaluateData['category'] == 'BUSINESS') & (travelAndBusinessEvaluateData['Prediction'] == 'BUSINESS')]
correctBusiness = (correctBusiness.all(axis='columns')).sum()
allOfBusiness = (travelAndBusinessEvaluateData['category'] == 'BUSINESS').sum()
allBusinessPrediction = (travelAndBusinessEvaluateData['Prediction'] == 'BUSINESS').sum()
BusinessRecall = correctBusiness/allOfBusiness
BusinessPrecision = correctBusiness/allBusinessPrediction
print(BusinessRecall,BusinessPrecision)
travelAndBusinessEvaluateData['correct'] = (travelAndBusinessEvaluateData['category'] == travelAndBusinessEvaluateData['Prediction'])
correctDetected = (travelAndBusinessEvaluateData['correct']).sum()
accuracy =  correctDetected / len(travelAndBusinessEvaluateData)
print(accuracy)

0.8691011235955056 0.8082549634273772
0.7380729653882133 0.8438502673796792
0.8504738504738505


| phase1 | Travel  | Business  |
| --- | --- | --- |
| Recall |
| Precision |
| Accuracy   |  |  |

##  Part $\mathrm{II}$

Now we want to add third category to our model. we add STYLE & BEAUTY category to our train set and repeat all the process again. 

In [15]:
sbWords = trainSB.words.values
sbWords = list(itertools.chain.from_iterable(sbWords))
allTrainDataWords = list(map(''.join, set(itertools.chain(travelWords, BusinessWords,sbWords))))
print(len(travelWords),len(BusinessWords),len(sbWords),len(allTrainDataWords))
sbWordsCount = dict(collections.Counter(sbWords))
travelWordsCount = dict(collections.Counter(travelWords))
BusinessWordsCount = dict(collections.Counter(BusinessWords))
print(len(travelWordsCount),len(BusinessWordsCount),len(sbWordsCount),len(allTrainDataWords))
allDataFrame = pd.DataFrame(columns=['Word','Travel occurrences','Business occurrences','Style & Beauty occurrences'])
for word in allTrainDataWords:
    to = 0
    sbo = 0
    bo = 0
    if word in travelWordsCount:
        to = travelWordsCount[word]
    if word in BusinessWordsCount:
        bo = BusinessWordsCount[word]
    if word in sbWordsCount:
        sbo = sbWordsCount[word]
    allDataFrame = allDataFrame.append({'Word' : word,'Travel occurrences' : to,'Business occurrences' : bo,'Style & Beauty occurrences' : sbo},ignore_index=True)


103034 52298 86274 22282
14426 8749 11301 22282


We also use Laplace smoothing here too.

In [16]:
alpha = 0.5
allDataFrame['Travel Probability'] = (allDataFrame['Travel occurrences'] + alpha) / (allDataFrame['Travel occurrences'].sum() + (len(set(travelWords + BusinessWords + sbWords))*alpha))
allDataFrame['Business Probability'] = (allDataFrame['Business occurrences'] + alpha) / (allDataFrame['Business occurrences'].sum() + (len(set(travelWords + BusinessWords + sbWords))*alpha))
allDataFrame['Style & Beauty Probability'] = (allDataFrame['Style & Beauty occurrences'] + alpha) / (allDataFrame['Style & Beauty occurrences'].sum() + (len(set(travelWords + BusinessWords + sbWords))*alpha))
allDataFrame = allDataFrame.set_index('Word')
allDataFrame.head()

,Travel occurrences,Business occurrences,Style & Beauty occurrences,Travel Probability,Business Probability,Style & Beauty Probability
Word,,,,,,
luminous,2,0,4,2.18962e-05,7.88159e-06,4.61941e-05
refinancers,0,2,0,4.37924e-06,3.94079e-05,5.13268e-06
tack,3,0,1,3.06547e-05,7.88159e-06,1.5398e-05
reload,0,0,1,4.37924e-06,7.88159e-06,1.5398e-05
puzzle,4,0,1,3.94132e-05,7.88159e-06,1.5398e-05


Now we can evaluate our model with our evaluation set which has all three category.

In [17]:
for index,row in evaluateData.iterrows():
    travelPriorProbability = len(trainTravel)/(len(trainTravel)+len(trainBusiness)+len(trainSB))
    businessPriorProbability = len(trainBusiness)/(len(trainTravel)+len(trainBusiness)+len(trainSB))
    sbPriorProbability = len(trainSB)/(len(trainTravel)+len(trainBusiness)+len(trainSB))
    words = set(row['words'])
    for word in words:
        if word in allTrainDataWords:
            travelPriorProbability *= allDataFrame.at[word,'Travel Probability']
            businessPriorProbability *= allDataFrame.at[word,'Business Probability']
            sbPriorProbability *= allDataFrame.at[word,'Style & Beauty Probability']
    evaluateData.at[index,'Travel Probability'] = travelPriorProbability
    evaluateData.at[index,'Business Probability'] = businessPriorProbability
    evaluateData.at[index,'Style & Beauty Probability'] = sbPriorProbability
    if travelPriorProbability >= businessPriorProbability and travelPriorProbability >= sbPriorProbability:
        evaluateData.at[index,'Prediction'] = 'TRAVEL'
    if businessPriorProbability >= travelPriorProbability and businessPriorProbability >= sbPriorProbability:
        evaluateData.at[index,'Prediction'] = 'BUSINESS'
    if sbPriorProbability >= travelPriorProbability and sbPriorProbability>= businessPriorProbability:
        evaluateData.at[index,'Prediction'] = 'STYLE & BEAUTY'


In [18]:
evaluateData.head()

,index,authors,category,date,headline,link,short_description,words,Travel Probability,Business Probability,Style & Beauty Probability,Prediction
5492,5492,Amanda McGowan,STYLE & BEAUTY,2012-11-20,"Iman Looking Fierce, Long Before David Bowie (...",https://www.huffingtonpost.com/entry/iman-davi...,Apparently not.,[apparently],5.609127e-05,3.122637e-05,5.249641e-05,TRAVEL
2211,2211,"Catherine Garvin, Contributor\nWriter, Author,...",STYLE & BEAUTY,2014-03-14,"Shear Madness TV Personality, Natalie Redding ...",https://www.huffingtonpost.com/entry/shear-mad...,Boho chic Natalie wears white linen pants by J...,"[boho, chic, natalie, wear, white, linen, pant...",2.260183e-72,4.416763e-74,1.555949e-64,STYLE & BEAUTY
15312,15312,Renee Jacques,STYLE & BEAUTY,2013-06-25,Why Shopping Malls Really Aren't Fun At All (P...,https://www.huffingtonpost.com/entry/shopping-...,1. The place is just WAY too big. You basicall...,"[place, way, big, basically, workout, day, go,...",2.453837e-37,2.679207e-38,2.256868e-37,TRAVEL
1956,1956,"Meg Waite Clayton, Contributor\nbestselling au...",TRAVEL,2013-06-25,6 Weeks in the English Lakes #3: Beatrix Potte...,https://www.huffingtonpost.com/entry/6-weeks-i...,Many people come to the English Lakes to immer...,"[many, people, come, english, lakes, immerse, ...",3.579503e-67,2.000793e-73,1.524785e-75,TRAVEL
21266,21266,"Daniel Burrus, ContributorTechnology Futurist,...",BUSINESS,2014-10-09,Profiting From the Speed of Change,https://www.huffingtonpost.com/entry/profiting...,NaN,[],3.881346e-01,2.330552e-01,3.788102e-01,TRAVEL


In [19]:
#Travel
correctTravel = evaluateData.loc[(evaluateData['category'] == 'TRAVEL') & (evaluateData['Prediction'] == 'TRAVEL')]
correctTravel = (correctTravel.all(axis='columns')).sum()
allOfTravel = (evaluateData['category'] == 'TRAVEL').sum()
allTravelPrediction = (evaluateData['Prediction'] == 'TRAVEL').sum()
travelRecall = correctTravel/allOfTravel
travelPrecision = correctTravel/allTravelPrediction
print(travelRecall,travelPrecision)
#Business
correctBusiness = evaluateData.loc[(evaluateData['category'] == 'BUSINESS') & (evaluateData['Prediction'] == 'BUSINESS')]
correctBusiness = (correctBusiness.all(axis='columns')).sum()
allOfBusiness = (evaluateData['category'] == 'BUSINESS').sum()
allBusinessPrediction = (evaluateData['Prediction'] == 'BUSINESS').sum()
BusinessRecall = correctBusiness/allOfBusiness
BusinessPrecision = correctBusiness/allBusinessPrediction
print(BusinessRecall,BusinessPrecision)
#Style
correctSB = evaluateData.loc[(evaluateData['category'] == 'STYLE & BEAUTY') & (evaluateData['Prediction'] == 'STYLE & BEAUTY')]
correctSB = (correctSB.all(axis='columns')).sum()
allOfSB = (evaluateData['category'] == 'STYLE & BEAUTY').sum()
allSBPrediction = (evaluateData['Prediction'] == 'STYLE & BEAUTY').sum()
SBRecall = correctSB/allOfSB
SBPrecision = correctSB/allSBPrediction
print(SBRecall,SBPrecision)

evaluateData['correct'] = (evaluateData['category'] == evaluateData['Prediction'])
correctDetected = (evaluateData['correct']).sum()
accuracy =  correctDetected / len(evaluateData)
print(accuracy)

0.8247191011235955 0.742914979757085
0.7128157156220767 0.7650602409638554
0.8480138169257341 0.912639405204461
0.8264282599215003


| phase1 | Travel  | Business | Style & Beauty |
| --- | --- | --- | --- |
| Recall |
| Precision |
| Accuracy   |  |  |

## Final Evaluation 

We have a dataset (test.csv) which is similar to our train dataset but has no category. we want to predict the category of news for this dataset with our trained model.

In [52]:
testDataFrame = pd.read_csv('Attachment/test.csv')
testDataFrame = cleanText(testDataFrame)
testDataFrame = testDataFrame.dropna(subset=['short_description'])
for index,row in testDataFrame.iterrows():
    travelPriorProbability = len(trainTravel)/(len(trainTravel)+len(trainBusiness)+len(trainSB))
    businessPriorProbability = len(trainBusiness)/(len(trainTravel)+len(trainBusiness)+len(trainSB))
    sbPriorProbability = len(trainSB)/(len(trainTravel)+len(trainBusiness)+len(trainSB))
    words = set(row['words'])
    for word in words:
        if word in allTrainDataWords:
            travelPriorProbability *= allDataFrame.at[word,'Travel Probability']
            businessPriorProbability *= allDataFrame.at[word,'Business Probability']
            sbPriorProbability *= allDataFrame.at[word,'Style & Beauty Probability']
    testDataFrame.at[index,'Travel Probability'] = travelPriorProbability
    testDataFrame.at[index,'Business Probability'] = businessPriorProbability
    testDataFrame.at[index,'Style & Beauty Probability'] = sbPriorProbability
    if travelPriorProbability >= businessPriorProbability and travelPriorProbability >= sbPriorProbability:
        testDataFrame.at[index,'category'] = 'TRAVEL'
    if businessPriorProbability >= travelPriorProbability and businessPriorProbability >= sbPriorProbability:
        testDataFrame.at[index,'category'] = 'BUSINESS'
    if sbPriorProbability >= travelPriorProbability and sbPriorProbability>= businessPriorProbability:
        testDataFrame.at[index,'category'] = 'STYLE & BEAUTY'

Now we write the category prediction into the ouptut.csv file.

In [55]:
output = pd.DataFrame({"index": testDataFrame['index'],"category": testDataFrame['category']})
output.to_csv('output.csv', index=False)
output

,index,category
0,0,STYLE & BEAUTY
1,1,TRAVEL
4,4,STYLE & BEAUTY
5,5,TRAVEL
6,6,STYLE & BEAUTY
...,...,...
2543,2543,BUSINESS
2544,2544,TRAVEL
2545,2545,BUSINESS
2546,2546,TRAVEL


## Questions

### 1

Stemming is the process of reducing inflection in words to their root forms such as mapping a group of words to the same stem even if the stem itself is not a valid word in the Language. On the other hand, Lemmatization reduces the inflected words properly ensuring that the root word belongs to the language. The result of Lemmatization is called lemma which is dictionary form, or citation form of the words. 
We use Lemmatization in our project because we interested in word frequency and want to treat all forms of a word as one. in stemming some forms of a word may not have the same stem and that can influence the frequency of that word and the accuracy of our model as well. 

### 2

tf-idf is short for "term frequency-inverse document frequency" which basically reflects how important a word is to a document.
tf measures how frequently a term occurs in a document.

TF = (Number of times term t appears in a document) / (Total number of terms in the document)
idf measures how important a term is.

IDF = $log_e$(Total number of documents / Number of documents with term t in it)

In Bayesian model, when we calculate the probabilities based on each word occurrences, each word in each document (in our case each news) counted as one. if we want to use tf-idf, instead of counting each word as one, we use the tf-idf weight.

### 3

We can get a hundred percent precision for certain a category even if our model cannot predict the categories correctly. for example, if we predict one TRAVEL news correctly and then assign all other news to other categories (e.g BUSINESS) then the precision of the TRAVEL will be a hundred percent although our predictions are wrong.

### 4

We referred to this problem earlier and the Laplace smoothing as a solution.